# Chapter 55: Service Virtualization

---

## 55.1 What is Service Virtualization?

Service virtualization is a technique used to simulate the behavior of specific components in a service-oriented architecture (SOA) or microservices environment. It creates a virtual version of a service that mimics the behavior of the real service, allowing teams to test their applications without relying on the actual service being available.

The virtual service (or "mock") responds to requests with predefined responses, emulating the real service's API, performance characteristics, and data. This enables development and testing to proceed even when the real service is under development, unstable, expensive to use, or inaccessible.

### 55.1.1 Key Concepts

- **Virtual Service (or API Mock):** A simulated service that stands in for a real dependency.
- **Contract:** Defines the expected request and response patterns.
- **Stubbing:** Providing canned responses for specific requests.
- **Record and Replay:** Capturing real interactions and replaying them in tests.
- **Stateful Virtualization:** Simulating services that maintain state across calls.

---

## 55.2 Why Use Service Virtualization?

| Benefit | Description |
|---------|-------------|
| **Parallel Development** | Frontend teams can work against virtual APIs while backend is still in development. |
| **Isolated Testing** | Test your service without dependencies on external systems that may be unreliable or slow. |
| **Cost Reduction** | Avoid costs associated with calling paid third-party APIs during testing. |
| **Scenario Simulation** | Easily simulate error conditions (timeouts, 500 errors) that are hard to reproduce with real services. |
| **Performance Testing** | Virtual services can be configured to simulate realistic latency and throughput. |
| **CI/CD Integration** | Virtual services can be spun up on demand in CI pipelines, ensuring tests are repeatable. |

---

## 55.3 When to Use Service Virtualization

- **Third-party APIs** (payment gateways, shipping providers, social media) – avoid rate limits and costs.
- **Legacy systems** that are difficult to set up in test environments.
- **Microservices still in development** – frontend can start early.
- **Mainframe or mainframe-based systems** – complex and expensive to access.
- **Batched or asynchronous processes** – simulate message queues.
- **Performance and load testing** – virtualize to focus on your system's behavior.

---

## 55.4 Service Virtualization Tools

| Tool | Language | Key Features |
|------|----------|--------------|
| **WireMock** | Java (standalone or library) | HTTP mocking, request matching, fault simulation, recording. |
| **Hoverfly** | Go (multi-language bindings) | HTTP/HTTPS, middleware, recording, simulation, middleware. |
| **Mountebank** | Node.js | Multi-protocol (HTTP, HTTPS, TCP, SMTP), imposter concept. |
| **MockServer** | Java, JavaScript | HTTP/HTTPS mocking, proxying, request verification. |
| **Betamax** | Groovy | Record/replay for JVM languages. |
| **VCR (Ruby)** | Ruby | Record HTTP interactions and replay in tests. |
| **Pact** | Multiple | Contract testing also provides mocks for consumers. |
| **Postman Mock Server** | Cloud-based | Mock APIs based on Postman collections. |

---

## 55.5 WireMock

WireMock is a popular library for HTTP-based service virtualization. It can run as a standalone server or embedded in tests.

### 55.5.1 Setup (Java with JUnit 5)

**Maven dependency:**

```xml
<dependency>
    <groupId>com.github.tomakehurst</groupId>
    <artifactId>wiremock-jre8</artifactId>
    <version>2.35.0</version>
    <scope>test</scope>
</dependency>
```

### 55.5.2 Basic Stubbing

```java
import com.github.tomakehurst.wiremock.junit5.WireMockExtension;
import static com.github.tomakehurst.wiremock.client.WireMock.*;
import static org.junit.jupiter.api.Assertions.assertEquals;

import org.junit.jupiter.api.Test;
import org.junit.jupiter.api.extension.RegisterExtension;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;

class WireMockTest {
    @RegisterExtension
    static WireMockExtension wireMock = WireMockExtension.newInstance()
        .options(options().port(8080))
        .build();

    @Test
    void testGetUser() throws Exception {
        // Arrange: stub the service
        wireMock.stubFor(get(urlEqualTo("/users/1"))
            .willReturn(aResponse()
                .withHeader("Content-Type", "application/json")
                .withBody("{\"id\":1,\"name\":\"John Doe\"}")));

        // Act: call the virtual service
        HttpClient client = HttpClient.newHttpClient();
        HttpRequest request = HttpRequest.newBuilder()
            .uri(URI.create("http://localhost:8080/users/1"))
            .build();
        HttpResponse<String> response = client.send(request, HttpResponse.BodyHandlers.ofString());

        // Assert
        assertEquals(200, response.statusCode());
        assertEquals("{\"id\":1,\"name\":\"John Doe\"}", response.body());
    }
}
```

### 55.5.3 Request Matching

WireMock can match requests based on URL, method, headers, body, etc.

```java
wireMock.stubFor(post(urlEqualTo("/orders"))
    .withHeader("Content-Type", equalTo("application/json"))
    .withRequestBody(containing("\"productId\": 123"))
    .willReturn(aResponse()
        .withStatus(201)
        .withHeader("Location", "/orders/456")));
```

### 55.5.4 Simulating Faults

```java
// Simulate a 500 error
wireMock.stubFor(get("/faulty")
    .willReturn(serverError()));

// Simulate timeout
wireMock.stubFor(get("/timeout")
    .willReturn(aResponse().withFixedDelay(5000)));
```

### 55.5.5 Recording and Proxying

WireMock can act as a proxy to record real interactions and then replay them.

```java
// Record mode
wireMock.stubFor(any(anyUrl())
    .willReturn(aResponse().proxiedFrom("https://real-api.example.com")));
```

Later, you can capture the recorded interactions and use them as stubs.

### 55.5.6 WireMock in Other Languages

- **Python:** `wiremock-python` library (wrapper around standalone).
- **JavaScript:** `wiremock` npm package (also wrapper).
- **Standalone:** Run as a Docker container: `wiremock/wiremock`.

---

## 55.6 Hoverfly

Hoverfly is a lightweight service virtualization tool written in Go. It supports HTTP, HTTPS, and can act as a middleware proxy.

### 55.6.1 Setup (Python Example)

```bash
pip install hoverfly
```

Hoverfly runs as a separate process; the Python client manages it.

### 55.6.2 Basic Simulation

```python
from hoverpy import Hoverfly

# Start Hoverfly in simulation mode
with Hoverfly(simulation="simulation.json") as hf:
    # Make requests to the virtual service
    response = requests.get("http://localhost:8500/api/users/1")
    print(response.json())
```

**Simulation file (`simulation.json`):**

```json
{
  "data": {
    "pairs": [
      {
        "request": {
          "path": [
            {
              "matcher": "exact",
              "value": "/api/users/1"
            }
          ],
          "method": [
            {
              "matcher": "exact",
              "value": "GET"
            }
          ]
        },
        "response": {
          "status": 200,
          "body": "{\"id\":1,\"name\":\"John Doe\"}",
          "headers": {
            "Content-Type": ["application/json"]
          }
        }
      }
    ]
  }
}
```

### 55.6.3 Recording Mode

```python
from hoverpy import Hoverfly

with Hoverfly(mode="capture") as hf:
    # Real requests are proxied through Hoverfly and recorded
    response = requests.get("https://real-api.example.com/users/1")
    # After this block, Hoverfly saves the captured interactions.
    # You can export them to a simulation file.
```

### 55.6.4 Middleware

Hoverfly can modify requests/responses on the fly using Lua scripts or external commands.

```lua
-- middleware.lua
if request.path == "/api/users/1" then
    response.body = "{\"id\":1,\"name\":\"Modified\"}"
end
```

```python
with Hoverfly(middleware="lua:middleware.lua") as hf:
    # Requests will be modified.
```

### 55.6.5 Java Client for Hoverfly

Hoverfly also has a Java client.

```java
import io.specto.hoverfly.junit5.HoverflyExtension;
import io.specto.hoverfly.junit5.api.HoverflyConfig;
import io.specto.hoverfly.junit5.api.HoverflySimulate;

@ExtendWith(HoverflyExtension.class)
@HoverflySimulate(source = @Source("simulation.json"))
class HoverflyTest {
    @Test
    void testGetUser() {
        // Hoverfly will simulate the response
        // Use any HTTP client to call localhost:8500
    }
}
```

---

## 55.7 Mountebank

Mountebank is a multi-protocol service virtualization tool. It uses the concept of "imposters" – simulated services that listen on a port and respond to requests.

### 55.7.1 Installation

```bash
npm install -g mountebank
```

### 55.7.2 Creating an Imposter (HTTP)

```bash
mb start  # starts the mountebank server
```

Then use its REST API to create imposters.

```bash
curl -X POST http://localhost:2525/imposters \
  -H "Content-Type: application/json" \
  -d '{
    "port": 3000,
    "protocol": "http",
    "stubs": [{
      "responses": [{
        "is": {
          "statusCode": 200,
          "headers": { "Content-Type": "application/json" },
          "body": { "id": 1, "name": "John Doe" }
        }
      }],
      "predicates": [{
        "equals": { "method": "GET", "path": "/users/1" }
      }]
    }]
  }'
```

Now `curl http://localhost:3000/users/1` returns the stubbed response.

### 55.7.3 Protocol Support

Mountebank supports HTTP, HTTPS, TCP, and SMTP, making it versatile for testing non-HTTP services.

---

## 55.8 Best Practices

### 55.8.1 Keep Virtual Services Simple

Virtual services should mimic the real service's behavior, not its implementation complexity. Focus on the contract.

### 55.8.2 Version Your Simulations

Store simulation definitions (JSON files) in version control alongside your tests. This ensures repeatability.

### 55.8.3 Use Record and Replay Wisely

Recording real interactions is great for creating realistic simulations, but be careful with sensitive data. Sanitize before committing.

### 55.8.4 Simulate Realistic Scenarios

Include edge cases: timeouts, errors, slow responses, large payloads. This tests your application's robustness.

### 55.8.5 Manage Lifecycle in CI/CD

Spin up virtual services in your CI pipeline before tests run, and shut them down afterward. Tools like Docker make this easy.

### 55.8.6 Avoid Over-Mocking

Too many virtualized services can hide real integration issues. Use contract tests to verify compatibility with real services, and only virtualize for isolation.

---

## 55.9 Common Challenges and Solutions

| Challenge | Solution |
|-----------|----------|
| **Virtual service diverges from real service** | Regularly update simulations based on real interactions; use contract testing. |
| **Performance mismatch** | Simulate realistic latency; use performance testing with real services periodically. |
| **Stateful interactions** | Use tools that support state (e.g., WireMock with scenarios, custom middleware). |
| **Authentication/authorization** | Virtualize auth endpoints with proper tokens; use same logic as real. |
| **Complex request matching** | Use advanced matching (regex, JSONPath, XPath). |

---

## Chapter Summary

In this chapter, we explored **Service Virtualization**:

- **What it is** – simulating dependent services to enable testing.
- **Why use it** – parallel development, isolation, cost savings, scenario simulation.
- **When to use it** – for third-party APIs, legacy systems, microservices under development.
- **Tools** – WireMock, Hoverfly, Mountebank, with detailed examples.
- **Best practices** – keep it simple, version simulations, use recording, simulate realistic scenarios.
- **Common challenges** and solutions.

**Key Insight:** Service virtualization is a powerful technique to remove dependencies and enable fast, reliable testing. Combined with contract testing, it ensures that your application works correctly with its dependencies, both real and simulated.

---

## 📖 Next Chapter: Chapter 56 - AI and Machine Learning in Testing

Now that you can virtualize services, Chapter 56 explores the cutting edge: **AI and Machine Learning in Testing**, covering test generation, maintenance, prediction, and the future of automated testing.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='54. chaos_engineering.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='56. ai_and_machine_learning_in_testing.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
